In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 500)
#pd.set_option('display.width', 1000)

In [ ]:
df = pd.read_csv('../input/house-prices-advanced-regression-techniques/train.csv')

In [ ]:
df.head()

In [ ]:
df.isnull().sum()

In [ ]:
df['KitchenAbvGr'].value_counts()

In [ ]:
sns.heatmap(df.isnull(), yticklabels=False, cbar=False)

In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
df['LotFrontage'].mean()

In [ ]:
# fill missing values
df['LotFrontage'] = df['LotFrontage'].fillna(df['LotFrontage'].mean())

In [ ]:
df.drop(['Alley'], axis=1, inplace=True)

In [ ]:
# all the null values of categorical features are replaced by mode values.
df['BsmtCond'] = df['BsmtCond'].fillna(df['BsmtCond'].mode()[0])
df['BsmtQual'] = df['BsmtQual'].fillna(df['BsmtQual'].mode()[0])
df['FireplaceQu'] = df['FireplaceQu'].fillna(df['FireplaceQu'].mode()[0])
df['GarageType'] = df['GarageType'].fillna(df['GarageType'].mode()[0])

In [ ]:
# all the features with more than 50% null values are dropped.
df.drop(['GarageYrBlt', 'PoolQC', 'Fence', 'MiscFeature'], axis=1, inplace=True)

In [ ]:
# all the NaN values categorical features are replaced by mode values.
df['GarageFinish'] = df['GarageFinish'].fillna(df['GarageFinish'].mode()[0])
df['GarageQual'] = df['GarageQual'].fillna(df['GarageQual'].mode()[0])
df['GarageCond'] = df['GarageCond'].fillna(df['GarageCond'].mode()[0])

In [ ]:
df.shape

In [ ]:
df.drop(['Id'], axis=1, inplace=True)

In [ ]:
df.isnull().sum()

In [ ]:
df['MasVnrType'] = df['MasVnrType'].fillna(df['MasVnrType'].mode()[0])
df['MasVnrArea'] = df['MasVnrArea'].fillna(df['MasVnrArea'].mode()[0])

In [ ]:
sns.heatmap(df.isnull(), yticklabels=False, cbar=False, cmap='coolwarm')

In [ ]:
df['BsmtExposure'] = df['BsmtExposure'].fillna(df['BsmtExposure'].mode()[0])

In [ ]:
sns.heatmap(df.isnull(), yticklabels=False, cbar=False, cmap='YlGnBu')

In [ ]:
df.dropna(inplace=True)

In [ ]:
df.shape

In [ ]:
df.head()

In [ ]:
##Handle Categorical Features

In [ ]:
columns = ['MSZoning','Street','LotShape','LandContour','Utilities','LotConfig','LandSlope','Neighborhood',
         'Condition2','BldgType','Condition1','HouseStyle','SaleType',
        'SaleCondition','ExterCond',
         'ExterQual','Foundation','BsmtQual','BsmtCond','BsmtExposure','BsmtFinType1','BsmtFinType2',
        'RoofStyle','RoofMatl','Exterior1st','Exterior2nd','MasVnrType','Heating','HeatingQC',
         'CentralAir',
         'Electrical','KitchenQual','Functional',
         'FireplaceQu','GarageType','GarageFinish','GarageQual','GarageCond','PavedDrive']

In [ ]:
len(columns)

In [ ]:
def category_onehot_multcols(multcolumns):
    df_final = final_df
    i = 0
    for fields in multcolumns:
        
        print(fields)
        df1 = pd.get_dummies(final_df[fields], drop_first=True)
        
        final_df.drop([fields], axis=1, inplace=True)
        if i==0:
            df_final = df1.copy()
        else:
            df_final = pd.concat([df_final, df1], axis=1)
        i=i+1
        
    df_final = pd.concat([final_df, df_final], axis=1)

    return df_final

In [ ]:
main_df = df.copy()

In [ ]:
## combine test data
test_df = pd.read_csv('../input/formulatedtest1/formulatedtest(1).csv')

In [ ]:
test_df.shape

In [ ]:
test_df.head()

In [ ]:
# combine train and test dataset row-wise
final_df = pd.concat([df, test_df], axis=0)

In [ ]:
final_df.shape

In [ ]:
final_df = category_onehot_multcols(columns)

In [ ]:
final_df.shape

In [ ]:
final_df = final_df.loc[:, ~final_df.columns.duplicated()]

In [ ]:
final_df.shape

In [ ]:
final_df.head()

In [ ]:
df_Train = final_df.iloc[:1421, :]
df_Test = final_df.iloc[1421:, :]

In [ ]:
X_train = df_Train.drop(['SalePrice'], axis=1)
y_train = df_Train['SalePrice']

In [ ]:
import xgboost
#classifier = xgboost.XGBRegressor()
#classifier.fit(X_train, y_train)
regressor = xgboost.XGBRegressor()

In [ ]:
n_estimators = [100, 500, 900, 1100, 1500]
max_depth = [2, 3, 5, 10, 15]
booster = ['gbtree', 'gblinear']
learning_rate = [0.05,0.1,0.15,0.20]
min_child_weight = [1,2,3,4]
base_score = [0.25, 0.5, 0.75, 1]

# define the grid of hyperparameters to search
hyperparameter_grid = {
    'n_estimators' : n_estimators,
    'max_depth' : max_depth,
    'learning_rate' : learning_rate,
    'min_child_weight' : min_child_weight,
    'booster' : booster,
    'base_score' : base_score
}

In [ ]:
# set up the random search with 4-fold cross validation
from sklearn.model_selection import RandomizedSearchCV
random_cv = RandomizedSearchCV(estimator=regressor,
                              param_distributions=hyperparameter_grid,
                              cv=5, n_iter=50,
                              scoring = 'neg_mean_absolute_error', n_jobs=4,
                              verbose=5,
                              return_train_score = True,
                              random_state  = 42)

In [ ]:
random_cv.fit(X_train, y_train)

In [ ]:
random_cv.best_estimator_

In [ ]:
regressor = xgboost.XGBRegressor(base_score=0.25, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.1, max_delta_step=0, max_depth=2,
             min_child_weight=1, missing=None, monotone_constraints='()',
             n_estimators=900, n_jobs=0, num_parallel_tree=1, random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=None)

In [ ]:
regressor.fit(X_train, y_train)

In [ ]:
df_Test.drop(['SalePrice'], axis=1, inplace=True)

In [ ]:
df_Test.shape

In [ ]:
import pickle
filename = 'finalized_model.pkl'

In [ ]:
y_pred = regressor.predict(df_Test)

In [ ]:
y_pred

In [ ]:
pred = pd.DataFrame(y_pred)
sub_df = pd.read_csv('../input/house-prices-advanced-regression-techniques/sample_submission.csv')

In [ ]:
datasets = pd.concat([sub_df['Id'], pred], axis=1)


In [ ]:
datasets.columns=['Id', 'SalePrice']


In [ ]:
datasets

In [ ]:
datasets.to_csv('sample_submission.csv', index=False)